<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">M0.532 · Pattern Recognition</p>
<p style="margin: 0; text-align:right;">Computational Engineering and Mathematics Master</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Computers, Multimedia and Telecommunications Department</p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>

In this notebook, we will see two different implementations of optical flow available in OpenCV. The first implementation is a sparse optical flow, which means that the optical flow is only computed for some pixels of the frames. The second implementation is a dense optical flow, which means that the optical flow is computed for all the pixels of the frames. Computing the optical flow can be useful for other video tasks such as object tracking or video object segmentation that we will see in the next notebooks. The code is based on [this tutorial](https://docs.opencv.org/4.x/d4/dee/tutorial_optical_flow.html).

#### Sparse Optical Flow

Let's start with the sparse optical flow implementation. We first download the video that we want to use for computing the optical flow.

In [ ]:
!wget https://www.bogotobogo.com/python/OpenCV_Python/images/mean_shift_tracking/slow_traffic_small.mp4

--2021-12-17 20:19:11--  https://www.bogotobogo.com/python/OpenCV_Python/images/mean_shift_tracking/slow_traffic_small.mp4
Resolving www.bogotobogo.com (www.bogotobogo.com)... 173.254.30.214
Connecting to www.bogotobogo.com (www.bogotobogo.com)|173.254.30.214|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2018126 (1.9M) [video/mp4]
Saving to: ‘slow_traffic_small.mp4’

slow_traffic_small. 100%[===================>]   1.92M  1.75MB/s    in 1.1s    

2021-12-17 20:19:13 (1.75 MB/s) - ‘slow_traffic_small.mp4’ saved [2018126/2018126]



First, we open the video file with the VideoCapture function from OpenCV. Since this is an sparse implementation, we need to decide some local feature detection parameters that will be used such as the maximum number of keypoints (maxCorners) or the minumum distance between two keypoints detected (minDistance). We also need to define some parameters for computing the optical flow by using the Lucas Kanade algorithm, such as the winSize, which defines the neighbouring region around a keypoint considered to compute the optical flow, or the maxLevel, which allows to compute the optical flow at different image scales.

The method is based on the detection of the local features by using the goodFeaturesToTrack function and then the computation of the optical flow by using the calcOpticalFlowPyrLK function.

In [ ]:
import numpy as np
import cv2 as cv
from google.colab.patches import cv2_imshow

cap = cv.VideoCapture('slow_traffic_small.mp4')
# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )
# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15, 15),
                  maxLevel = 2,
                  criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))
# Create some random colors
color = np.random.randint(0, 255, (100, 3))
# Take first frame and find corners in it
ret, old_frame = cap.read()
old_gray = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)
p0 = cv.goodFeaturesToTrack(old_gray, mask = None, **feature_params)
# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)
while(1):
    ret, frame = cap.read()
    if not ret:
        print('No frames grabbed!')
        break
    frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    # calculate optical flow
    p1, st, err = cv.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
    # Select good points
    if p1 is not None:
        good_new = p1[st==1]
        good_old = p0[st==1]
    # draw the tracks
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel()
        c, d = old.ravel()
        mask = cv.line(mask, (int(a), int(b)), (int(c), int(d)), color[i].tolist(), 2)
        frame = cv.circle(frame, (int(a), int(b)), 5, color[i].tolist(), -1)
    img = cv.add(frame, mask)
    cv2_imshow(img)
    k = cv.waitKey(30) & 0xff
    if k == 27:
        break
    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1, 1, 2)


####Dense Optical Flow

In order to compute the dense optical flow, we follow a similar implementation, with a few changes. We don't need to call the goodFeaturesToTrack function since we don't want to compute the optical flow based only on a few local features. Since we want to compute the optical flow for all the pixels of the frames, we will call the calcOpticalFlowFarneback function also available in OpenCV library. The color representation of the output in the HSV color space represents both the direction and the magnitude of the optical flow.

In [ ]:
import numpy as np
import cv2 as cv
from google.colab.patches import cv2_imshow
cap = cv.VideoCapture('slow_traffic_small.mp4')
ret, frame1 = cap.read()
prvs = cv.cvtColor(frame1, cv.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame1)
hsv[..., 1] = 255
while(1):
    ret, frame2 = cap.read()
    if not ret:
        print('No frames grabbed!')
        break
    next = cv.cvtColor(frame2, cv.COLOR_BGR2GRAY)
    flow = cv.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    mag, ang = cv.cartToPolar(flow[..., 0], flow[..., 1])
    hsv[..., 0] = ang*180/np.pi/2
    hsv[..., 2] = cv.normalize(mag, None, 0, 255, cv.NORM_MINMAX)
    bgr = cv.cvtColor(hsv, cv.COLOR_HSV2BGR)
    cv2_imshow(bgr)
    k = cv.waitKey(30) & 0xff
    if k == 27:
        break
    elif k == ord('s'):
        cv.imwrite('opticalfb.png', frame2)
        cv.imwrite('opticalhsv.png', bgr)
    prvs = next
